In [ ]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
#Read resources
CA_videos = pd.read_csv('Resources/CAvideos.csv')
US_videos = pd.read_csv('Resources/USvideos.csv')
CA_category_id = pd.read_json('Resources/CA_category_id.json')
US_category_id = pd.read_json('Resources/US_category_id.json')

In [ ]:
#Jumaan start here

In [ ]:
#Check the columns on US_videos
US_videos.head()

In [ ]:
#Drop unnecessary columns
US_videos_new = US_videos[['title', 'channel_title', 'category_id', 'views', 'likes', 'dislikes', 'comment_count']]
US_videos_new.head()

In [ ]:
#Check for duplicates and drop any duplicate rows
print(len(US_videos_new)) 
print(len(US_videos_new['title']))

In [ ]:
#Check the columns on US_category_id
US_category_id.head()

In [ ]:
#Turn items column into a list
US_category_id_list = US_category_id['items'].tolist()

#Loop through the list and extract category id and title
category_id = []
category_title = []

for category in US_category_id_list:
    category_id.append(int(category['id']))
    category_title.append(category['snippet']['title'])

#Store the information in a datagrame
US_categories = pd.DataFrame({"category_title": category_title, "category_id": category_id})

US_categories.head()

In [ ]:
#Merge both dataframes on category_id
US_merged = pd.merge(US_videos_new, US_categories, on="category_id", how="inner")
US_merged.head()

In [ ]:
#Jumaan end here

In [ ]:
#Shrey start here

In [ ]:
#Shrey end here

In [ ]:
#Anji start here

In [ ]:
#Anji end here